In [2]:
%pip install opencv-python mediapipe tensorflow numpy


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached numpy-2.3.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.8.0-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.0-cp312-cp312-win_amd64.whl.metadata (1.4 kB)
  Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata (11 kB)
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl.metadata (9.

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\HP\\OneDrive\\Desktop\\Air Writing\\.venv\\Lib\\site-packages\\PIL\\WalImageFile.py'
Check the permissions.



In [1]:
import tensorflow as tf
print("✅ TensorFlow version:", tf.__version__)


✅ TensorFlow version: 2.19.1


In [1]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.datasets import mnist # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D # type: ignore

# ========================
# Step 1: Prepare/load MNIST model
# ========================

try:
    model = load_model("mnist_model.h5")
    print("✅ Model loaded successfully.")
except:
    print("⚠️ Model not found. Training new model...")
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
    x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    model = Sequential([
        Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(64, kernel_size=(3,3), activation='relu'),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=15,
    verbose=1
)

    model.save("mnist_model.h5")
    print("✅ Model trained and saved.")

    

# ========================
# Step 2: Hand Tracking Setup
# ========================

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# ========================
# Step 3: Drawing Canvas
# ========================

canvas = np.zeros((512,512), dtype=np.uint8)
drawing = False
prev_x, prev_y = None, None

cap = cv2.VideoCapture(0)

print("🎯 Air-writing Digit Recognition Started. Press 'q' to quit, 'c' to clear canvas, 'p' to predict digit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
            x = int(handLms.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * 512)
            y = int(handLms.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * 512)
            if prev_x is None:
                prev_x, prev_y = x, y
            cv2.line(canvas, (prev_x, prev_y), (x, y), 255, 10)
            prev_x, prev_y = x, y
    else:
        prev_x, prev_y = None, None

    cv2.imshow("Air Canvas", canvas)
    cv2.imshow("Webcam", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        canvas = np.zeros((512,512), dtype=np.uint8)
    elif key == ord('p'):
        # Preprocess canvas and predict
        img = cv2.resize(canvas, (28,28))
        img = img.astype('float32') / 255.0
        img = img.reshape(1,28,28,1)
        pred = np.argmax(model.predict(img), axis=1)[0]
        print(f"🎉 Predicted Digit: {pred}")

cap.release()
cv2.destroyAllWindows()





✅ Model loaded successfully.
🎯 Air-writing Digit Recognition Started. Press 'q' to quit, 'c' to clear canvas, 'p' to predict digit.


c:\Users\HP\OneDrive\Desktop\Air Writing\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
from tensorflow.keras.datasets import mnist # type: ignore
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = x_test.reshape(-1,28,28,1)/255.0
y_test = to_categorical(y_test, 10)

# Load model
model = load_model("mnist_model.h5")

# Evaluate
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy)


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9910 - loss: 0.0309
Test Accuracy: 0.9909999966621399


In [4]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model # type: ignore
import time

model = load_model("mnist_model.h5")

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

canvas = np.zeros((480, 640, 3), dtype=np.uint8)
cap = cv2.VideoCapture(0)

digits_to_draw = [0,1,2,3,4,5,6,7,8,9]  # sequence of targets
current_digit = 0
correct = 0
total = 0
prev_x, prev_y = 0, 0
start_time = time.time()

print(f"Draw digit: {digits_to_draw[current_digit]}")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
            x = int(handLms.landmark[8].x * 640)
            y = int(handLms.landmark[8].y * 480)

            if prev_x == 0 and prev_y == 0:
                prev_x, prev_y = x, y
            cv2.line(canvas, (prev_x, prev_y), (x, y), (255, 255, 255), 8)
            prev_x, prev_y = x, y

    # Every 5 seconds, evaluate prediction
    if time.time() - start_time > 5:
        gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            cnt = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(cnt)
            digit = thresh[y:y+h, x:x+w]
            digit = cv2.resize(digit, (28,28))
            digit = digit.reshape(1,28,28,1)/255.0
            pred = model.predict(digit)
            y_pred = np.argmax(pred)

            # Compare prediction
            y_true = digits_to_draw[current_digit]
            total += 1
            if y_pred == y_true:
                correct += 1

            print(f"Expected: {y_true}, Predicted: {y_pred}")
            print(f"Real-time accuracy: {(correct/total)*100:.2f}%")

        # Prepare for next digit
        current_digit = (current_digit + 1) % len(digits_to_draw)
        print(f"Now draw: {digits_to_draw[current_digit]}")
        start_time = time.time()
        canvas = np.zeros((480, 640, 3), dtype=np.uint8)
        prev_x, prev_y = 0, 0

    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
    cv2.imshow("Air Writing Digit", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Draw digit: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Expected: 0, Predicted: 7
Real-time accuracy: 0.00%
Now draw: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Expected: 1, Predicted: 2
Real-time accuracy: 0.00%
Now draw: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Expected: 2, Predicted: 7
Real-time accuracy: 0.00%
Now draw: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Expected: 3, Predicted: 4
Real-time accuracy: 0.00%
Now draw: 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Expected: 4, Predicted: 2
Real-time accuracy: 0.00%
Now draw: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Expected: 5, Predicted: 1
Real-time accuracy: 0.00%
Now draw: 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Expected: 6, Predicted: 1
Real-time accuracy: 0.00%
Now draw: 7
Now draw: 8
Now draw: 9
Now draw: 0


KeyboardInterrupt: 